# 3D Rigid Body Motion

## Using Numpy/Scipy as a replacement for Eigen

In [1]:
import numpy as np

### Basic Matrix Ops

In [2]:
m = np.random.randn(3,3)
print(f"Transpose = {m.T}")
print(f"\nSum: {m.sum()}")
print(f"Trace: {m.trace()}")
print(f"\nScalar multiplication: {m * 10}")
print(f"\nInverse matrix: {np.linalg.inv(m)}") # <- Computed inverse using LU factorisation
print(f"\nDeterminant: {np.linalg.det(m)}")

Transpose = [[ 0.98463153  0.32513755 -0.05814333]
 [ 0.12867984 -0.0942763   1.22328457]
 [-1.35543483 -0.65870067  0.05531716]]

Sum: 0.5504955250412655
Trace: 0.9456723889838698

Scalar multiplication: [[  9.84631534   1.28679839 -13.55434834]
 [  3.25137553  -0.94276302  -6.58700672]
 [ -0.58143325  12.23284575   0.55317157]]

Inverse matrix: [[ 3.08860794 -6.42441685 -0.82001511]
 [ 0.07836984 -0.09391431  0.80198958]
 [ 1.51333383 -4.67581925 -0.51954755]]

Determinant: 0.25919873628127793


In [3]:
# Eigen values
s = m.T @ m # construct a real symmetric matrix: A^T A
eigVals, eigVecs = np.linalg.eigh(s) # `eigh` is optimised for symmetric matrices, compared to the generalist `eig`
print(f"Eigen Values: {eigVals}")
print(f"Eigen Vectors: {eigVecs}")

Eigen Values: [0.01322584 1.5204613  3.34092902]
Eigen Vectors: [[ 0.82441687  0.01841541  0.56568339]
 [ 0.00313031 -0.9996036   0.02797931]
 [ 0.5659744  -0.02129585 -0.82414772]]


In [4]:
# Solving Equations
sz = 100
A = np.random.randn(sz, sz)
A = A @ A.T # Make `A` into a positive semi-definite matrix (x^T.M.x >= 0) by representing it as a symmetric matrix
b = np.random.randn(sz)
# By positive semi-definiteness, `b.T @ A @ b` >= 0

In [5]:
# by direct inversion using LU factorisation
x = np.linalg.inv(A) * b
print(f"Condition number for A: {np.linalg.cond(A)}, (could be worse)")
print(np.linalg.norm(x))

Condition number for A: 110667.37408080988, (could be worse)
330.7890493425934


In [6]:
# QR decomposition
import scipy
def qr_solve(A, b):
    Q, R = np.linalg.qr(A)
    # Q is orthonormal, therefore, its transpose is also its inverse
    p = Q.T @ b
    Rinv = np.linalg.inv(R) # can't be bothered to solve directly using upper triangular R structure
    return Rinv @ p
    # return scipy.linalg.solve_triangular(R, p)

x = qr_solve(A, b)
print(np.linalg.norm(x)) # <- norm is much lower here, QR decomp is much more stable and accurate

264.77405108798933


In [7]:
# Cholesky decomp
L = np.linalg.cholesky(A)
y = np.linalg.solve(L, b)
x = np.linalg.solve(L.T, y)
print(np.linalg.norm(x))

264.7740510877372


## Replicating Eigen Geometry with Numpy/Scipy

In [8]:
from scipy.spatial.transform import Rotation as R

In [9]:
id_rot = R.identity()
print(id_rot.as_matrix())
print(id_rot.as_quat()) # scalar last format
print(id_rot.as_rotvec())

[[1. 0. 0.]
 [0. 1. 0.]
 [0. 0. 1.]]
[0. 0. 0. 1.]
[0. 0. 0.]


In [43]:
# Axis Angle Representation
rotvec = R.from_euler('z', np.pi/4) # 45 degree rotation along the z-axis
print(rotvec.as_matrix)

v = np.array([1, 0, 0])
v_rotated = rotvec.apply(v)
print(v_rotated)

[0.70710678 0.70710678 0.        ]


In [78]:
from Transforms import Transform

T = Transform.identity()

T.rotate(rotvec)
# T.pretranslate(np.array([1, 3, 4]))

v = np.array([1, 0, 0])
v_rot = T * v
print(v_rot)

[0.70710678 0.70710678 0.        ]


In [79]:
q = rotvec
v_rot = q.apply(v)
print(v_rot)

[0.70710678 0.70710678 0.        ]


In [ ]:
# Coordinate transformation example